Copyright 2020 DeepMind Technologies Limited

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Clone repo and import dependencies

In [1]:
!pip install tensorflow==1.15 dm-sonnet==1.36 tensor2tensor==1.14 pyntcloud 

import os
import numpy as np
import time
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)  # Hide TF deprecation messages
import matplotlib.pyplot as plt

%cd /tmp
%rm -rf /tmp/deepmind_research
!git clone https://github.com/davidharrod/PengLai.git
%cd /tmp/PengLai
import modules
import data_utils
import train
from pre_process import point_cloud_2_voxel as p2v

/tmp
正克隆到 'PMGen'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 39 (delta 5), reused 0 (delta 0), pack-reused 0
展开对象中: 100% (39/39), 完成.
[Errno 2] No such file or directory: '/tmp/PolyGen'
/tmp


## Prepare dataset

In [ ]:
batch_size = 1
buffer_size = 1
obj_path = "./dataset/obj/"
binvox_path = "./dataset/binvox"
vertex_dataset, face_dataset = train.load_dataset(obj_path, binvox_path,batch_size,buffer_size)

## Set configuration

In [ ]:
VERTICES = "VERTICES_FLAT"
FACES = "FACES"
EPOCH = 10
CHECK_POINT = 5

# Set tensorflow gpu configuration.
tf_gpu_config = tf.ConfigProto(allow_soft_placement=True)
tf_gpu_config.gpu_options.allow_growth = True

# Set encoders and decoders' configuration.
vertex_model_encoder_config = {
    "hidden_sizes": (64, 256),
    "num_blocks": (1, 2),
    "dropout_rate": 0.1
}
vertex_model_decoder_config = {
    "hidden_size": 256,
    "fc_size": 1024,
    "num_heads": 4,
    "dropout_rate": 0.2,
}
face_model_encoder_config = {
    'hidden_size': 128,
    'fc_size': 512,
    'num_layers': 3,
    'dropout_rate': 0.
}
face_model_decoder_config = {
    'hidden_size': 128,
    'fc_size': 512,
    'num_layers': 3,
    'dropout_rate': 0.
}

## Create models

In [ ]:
# Set training parameters.
target_dir = "./log/"
learning_rate = 5e-4
training_step = 500
check_step = 10

# Prepare for training.
vertex_batch = vertex_dataset.make_one_shot_iterator().get_next()
face_batch = face_dataset.make_one_shot_iterator().get_next()
_, vertex_loss, vertex_samples = train._create_model(
    vertex_batch,
    model_type=VERTICES,
    encoder_config=vertex_model_encoder_config,
    decoder_config=vertex_model_decoder_config)
_, face_loss, _=train._create_model(face_batch,
                                model_type=FACES,
                                encoder_config=face_model_encoder_config,
                                decoder_config=face_model_decoder_config,
                                vertex_samples=vertex_samples)
optimizer = tf.train.AdamOptimizer(learning_rate)
vertex_model_optim_op = optimizer.minimize(vertex_loss)
face_model_optim_op = optimizer.minimize(face_loss)
# Create directory for saving log and model.
current_time = time.strftime("%Y_%m_%d_%H_%M", time.localtime())
writer_path = p2v._try_2_create_directory(
    target_dir, f"{current_time}/tensorboard/graphs")
saver_path = p2v._try_2_create_directory(target_dir,
                                         f"{current_time}/ckpt_model")
# Set saver and summary.
saver = tf.train.Saver()
tf.summary.scalar("vertex loss", vertex_loss)
tf.summary.scalar("face loss", face_loss)
merged = tf.summary.merge_all()

## Start training

In [ ]:
with tf.Session(config=tf_gpu_config) as sess:
    writer = tf.summary.FileWriter(writer_path, sess.graph)
    sess.run(tf.global_variables_initializer())
    for epoch in range(EPOCH):
        for n in range(training_step):
            if n % check_step == 0:
                summary = sess.run(merged)
                writer.add_summary(summary, n + epoch * training_step)
            sess.run((vertex_model_optim_op, face_model_optim_op))
        if (epoch + 1) % CHECK_POINT == 0:
            saver.save(sess,
                       os.path.join(saver_path, "keypoint_model.ckpt"),
                       global_step=epoch)

## Check out logging with tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir writer_path